# Hyperparameter Tuning

Hyperparameters of machine learning algorithms are parameters set before training the model. These parameters affect model performance but cannot be learned through the normal training process. To identify good hyperparameters for a particular problem, we must build and test models with a variety of hyperparameters.

In [4]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import *

In [5]:
myseed = 1

## Building a Defect Prediction Model

In [7]:
# Apache Camel dataset
camel = pd.read_csv('data/camel-1.6.csv')
camel["defective"] = (camel["bug"] > 0).astype(int)
camel.drop(['name', 'version', 'name.1', 'bug'], axis=1, inplace=True)
camel.head()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,defective
0,5,3,0,7,10,0,1,7,4,0.250000,...,1.0,1,0.921053,0.360000,1,2,7.400,1,0.6000,0
1,4,1,0,3,5,4,1,2,3,0.666667,...,1.0,1,0.000000,0.500000,0,0,3.000,1,0.5000,0
2,20,4,0,26,95,144,2,26,13,0.842105,...,1.0,0,0.727273,0.197368,4,5,20.300,3,1.0000,0
3,3,2,0,8,22,3,2,6,2,2.000000,...,0.0,0,0.750000,0.666667,1,3,54.000,15,5.3333,1
4,8,1,0,25,20,22,22,3,6,0.571429,...,1.0,0,0.000000,0.250000,0,0,20.875,1,0.7500,1


In [9]:
# Distribution of neutral (0) and defective (1) files
camel["defective"].value_counts()

0    777
1    188
Name: defective, dtype: int64

In [10]:
# Split data into 80% training and 20% test sets
y = camel['defective'].values                  # y is response vector
X = camel.drop('defective', axis=1).values     # X is array of predictors(features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=myseed)

In [11]:
# Base model fitting and accuracy
lr_base = LogisticRegression(random_state=myseed)
lr_base.fit(X_train, y_train)

round(lr_base.score(X_test, y_test), 3)

/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.834

In [12]:
y_pred = lr_base.predict(X_test)
print(round(precision_score(y_test, y_pred), 3))
print(round(recall_score(y_test, y_pred), 3))
print(round(f1_score(y_test, y_pred), 3))
print(round(average_precision_score(y_test, y_pred), 3))
print(round(roc_auc_score(y_test, y_pred), 3))

0.417
0.167
0.238
0.199
0.562


## Grid Search

A grid search finds the best hyperparameters from a user specified list by building models for all combinations of hyperparameters in the list and selecting the model with best performance. Different model types have different hyperparameters.

[Logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) has parameters that control how much regularization is applied. Regularization penalizes too much complexity in linear models to reduce overfitting. It will not improve performance on the training data set, but it can improve performance on the test data set. There are two parameters:

  * penalty: controls the form of the regularization term in the equation: L2 is based on squares of the coefficients of the linear equation, while L1 is based on absolute values of those coefficients.
  * C: is the inverse of the lambda coefficient of the penalty term. Larger C values mean less regularization.

In [13]:
# Logistic Regression Grid
best_score = 0

for penalty in ['l1', 'l2']:
    for C in np.logspace(-4, 4, 10):
        model = LogisticRegression(penalty=penalty, C=C, random_state=myseed)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_parameters = {'penalty': penalty, 'C': C}
            
print("Best score: {:.3f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to '

Best score: 0.850
Best parameters: {'penalty': 'l1', 'C': 0.046415888336127774}


/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to '

### Random Forest

[Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) models have many hyperparameters. We cannot tune all of them due to the time it would require to build and evaluate so many models, but we can tune some of the most important hyperparameters, including:

  * **n_estimators**: Number of trees in the random forest.
  * **max_depth**: Maximum depth of trees to construct.
  * **max_features**: Maximum number of features considered when splitting a node.

In [14]:
# Base model fitting and accuracy
rf_base = LogisticRegression(random_state=myseed)
rf_base.fit(X_train, y_train)

round(rf_base.score(X_test, y_test), 3)

/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.834

In [15]:
# Random Forest Grid for n_estimator and max_depth
best_score = 0

for n_estimator in [10, 50, 100, 500, 1000]:
    for max_depth in [1, 2, 4]:
        model = RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, random_state=myseed)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_parameters = {'n_estimators': n_estimator, 'max_depth': max_depth}
            
print("Best score: {:.3f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

Best score: 0.850
Best parameters: {'n_estimators': 50, 'max_depth': 2}


### GridSearchCV

While it is easy enough to write our own grid searches using loops, scikit-learn provides a convenient function [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) that performs grid search using cross-validation, which should 

   * Provide a more accurate performance score for new data than using the test data multiple times.
   * Reduce the effect of different data splits on model performance. 

The disadvantage of using cross-validation is that it will increase the amount of computation required for the search.

In [16]:
# Grid of parameters to search
lr_param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-4, 4, 10)
}

In [17]:
# We'll use 5-fold cross validation in our grid search
grid_search = GridSearchCV(LogisticRegression(), lr_param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([1.00000e-04, 7.74264e-04, 5.99484e-03, 4.64159e-02, 3.59381e-01,
       2.78256e+00, 2.15443e+01, 1.66810e+02, 1.29155e+03, 1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [18]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))

Best parameters: {'C': 0.3593813663804626, 'penalty': 'l2'}
Best cross-validation score: 0.804


In [19]:
print("Test set score: {:.3f}".format(grid_search.score(X_test, y_test)))

Test set score: 0.834


In [23]:
# Grid of parameters to search
rf_param_grid = {
    'n_estimators': [10, 50, 100, 200, 500, 1000],
    'max_depth': [1, 2, 4, 8]
}

In [24]:
# We'll use 5-fold cross validation in our grid search
grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 50, 100, 200, 500, 1000], 'max_depth': [1, 2, 4, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [25]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))

Best parameters: {'max_depth': 4, 'n_estimators': 100}
Best cross-validation score: 0.808


In [26]:
print("Test set score: {:.3f}".format(grid_search.score(X_test, y_test)))

Test set score: 0.845


## Differential Evolution

There are too many possible hyperparameter configurations to try all of them, so we use differential evolution to explore the searchspace more efficiently than we can with grid or randomized searches. At each iteration in differential evolution, a population of candidate solutions is created by mixing candidate solutions from a previous generation (iteration.) The initial population of candidates is generated through a user-specified algorithm with an aim towards being spread across the entire searchspace. The number of candidates in each generation is also a parameter given to the algorithm.

Candidates are evaluated via a user-defined objective function, with lower values being better. For hyperparameter tuning, we create an objective function that returns the mean metric score obtained via cross-validation using the training data set. Higher metric scores are better and the optimizer is looking for minimum values, so the object function multiplies the mean score by -1 to return a lower value when model performance is better.

The search may produce some warnings about ill-defined metrics as some choices of hyperparameters lead to no true positives or negatives, causing errors when computing certain model performance metrics.

In [27]:
import scipy.optimize as opt

In [28]:
# Number of folds and metric for cross-validation
nfolds = 3
metric = 'f1'

# Define objective function with evolving parameters (taken from bounds) and fixed arguments (training data)
def hypertuner(parameters, *data):
    # Ensure parameters are treated as integers
    parameters = list(map(int,parameters))
    # Create the model
    model = RandomForestClassifier(
        n_estimators = parameters[0], 
        max_depth = parameters[1],
        max_features = parameters[2],
        random_state = myseed
    )
    # Perform cross validation using all cores
    cv_scores = cross_val_score(model, data[0], data[1], scoring=metric, cv=nfolds, n_jobs=-1)
    average_score = np.mean(cv_scores)
    
    print(np.round(average_score,3), parameters)
    return -1*average_score

In [29]:
# Parameter ranges to optimize random forest over
bounds = [
    (10,1000),  # n_estimators
    (1,20),      # max_depth
    (1,10),     # max_features
]
bounds

[(10, 1000), (1, 20), (1, 10)]

In [30]:
# Fixed arguments for objective function: the training data
mydata = (X_train, y_train)

In [31]:
%time result = opt.differential_evolution(hypertuner, bounds, mydata, strategy='rand2bin', \
                                      popsize=10, mutation=(0.5,1.9), recombination=0.7, maxiter=3)

0.072 [919, 3, 2]
0.334 [298, 13, 9]
0.324 [523, 15, 8]
0.311 [749, 13, 5]
0.223 [434, 5, 6]
0.136 [622, 4, 3]
0.212 [825, 6, 4]
0.323 [654, 17, 9]
0.339 [35, 12, 4]
0.251 [464, 8, 4]
0.0 [139, 1, 9]
0.307 [478, 14, 6]
0.27 [73, 15, 2]
0.127 [717, 2, 7]
0.06 [173, 4, 1]
0.315 [576, 12, 4]
0.19 [547, 10, 1]
0.258 [205, 9, 3]
0.318 [879, 19, 8]
0.307 [333, 16, 2]
0.22 [966, 6, 6]
0.291 [230, 10, 5]
0.238 [792, 7, 7]
0.012 [673, 1, 9]
0.319 [368, 18, 8]
0.264 [388, 17, 1]
0.205 [866, 8, 3]
0.293 [98, 11, 5]
0.329 [253, 19, 7]
0.176 [986, 5, 3]
0.231 [757, 12, 1]
0.237 [628, 7, 4]
0.315 [909, 15, 5]
0.309 [60, 13, 3]
0.328 [395, 11, 7]
0.308 [618, 18, 2]
0.249 [484, 8, 4]
0.145 [654, 8, 1]
0.187 [891, 3, 9]
0.278 [219, 9, 5]
0.0 [599, 1, 6]
0.229 [931, 5, 9]
0.049 [73, 2, 3]
0.213 [717, 8, 5]
0.253 [173, 15, 1]
0.311 [576, 13, 9]
0.248 [89, 8, 4]
0.246 [835, 15, 1]
0.308 [615, 11, 6]
0.332 [744, 19, 4]
0.0 [358, 2, 1]
0.291 [219, 10, 5]
0.0 [130, 1, 3]
0.256 [673, 13, 1]
0.318 [801, 12, 8]

In [32]:
result

     fun: -0.34009936766034327
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 124
     nit: 3
 success: False
       x: array([382.47634662,  15.68055743,   7.08571931])

In [33]:
# Best F1 value for any run
np.round(-result.fun, 3)

0.34

# To do on your own

In class, reproduce the hyperparameter tuning used in this week's paper on the CamelV0 (1.0, 1.2, 1.4) dataset. Use the hyperparameter ranges given in Table 2 of the paper. The data can be downloaded from [Seacraft](https://zenodo.org/communities/seacraft). The three datasets are to be used in the same way as the paper:

    * Training: train the models on the oldest (1.0) dataset.
    * Validation: evaluate the models within the objective function on the middle (1.2) dataset.
    * Testing: measure the final precision using the most recent (1.4) dataset.

How does your performance compare to that of the model in the paper? Do you find a large improvement in precision for the best tuned model compared to an untuned random forest model trained on the oldest dataset and evaluated on the testing dataset?

In [34]:
# Apache Camel dataset
camel = pd.read_csv('data/camel-1.0.csv')
camel["defective"] = (camel["bug"] > 0).astype(int)
camel.drop(['name', 'version', 'name.1', 'bug'], axis=1, inplace=True)
camel.head()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,defective
0,4,2,0,6,8,6,2,5,4,2.00,...,0.0,0,0.896552,0.500000,0,0,5.500000,1,0.7500,0
1,6,3,0,21,33,15,1,21,2,2.00,...,0.0,0,0.800000,0.500000,2,2,28.333333,1,0.6667,0
2,2,3,0,3,7,1,0,3,1,2.00,...,0.0,0,0.833333,0.666667,1,1,11.000000,1,0.5000,0
3,26,1,1,10,47,0,5,5,24,0.08,...,1.0,1,0.000000,0.258242,0,0,8.038462,2,1.0000,0
4,4,3,0,4,19,6,1,4,3,2.00,...,0.0,0,0.888889,0.375000,1,1,14.500000,1,0.5000,0


In [39]:
# Apache Camel dataset
camel2 = pd.read_csv('data/camel-1.2.csv')
camel2["defective"] = (camel2["bug"] > 0).astype(int)
camel2.drop(['name', 'version', 'name.1', 'bug'], axis=1, inplace=True)
camel2.head()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,defective
0,9,0,0,5,24,0,1,4,9,0.000000,...,1.0,1,0.000000,0.370370,0,0,22.555556,1,0.8889,0
1,27,1,0,6,27,351,1,5,27,2.000000,...,0.0,0,0.000000,0.412037,0,0,0.000000,1,1.0000,1
2,9,1,0,3,18,30,1,2,8,0.375000,...,1.0,0,0.000000,0.355556,0,0,8.000000,1,0.7778,0
3,7,1,0,27,50,7,3,27,3,0.805556,...,1.0,3,0.000000,0.416667,0,0,26.428571,1,0.7143,0
4,8,2,0,10,28,0,1,9,3,0.357143,...,1.0,3,0.684211,0.371429,1,1,26.500000,2,1.0000,1


In [35]:
# Distribution of neutral (0) and defective (1) files
camel["defective"].value_counts()

0    326
1     13
Name: defective, dtype: int64

In [36]:
# Split data into 80% training and 20% test sets
y = camel['defective'].values                  # y is response vector
X = camel.drop('defective', axis=1).values     # X is array of predictors(features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=myseed)

In [42]:
# Split data into 0% training and 100% test sets
y = camel2['defective'].values                  # y is response vector
X = camel2.drop('defective', axis=1).values     # X is array of predictors(features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=myseed)

In [43]:
# Base model fitting and accuracy
lr_base = LogisticRegression(random_state=myseed)
lr_base.fit(X_train, y_train)

round(lr_base.score(X_test, y_test), 3)

/home/student/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.633

In [44]:
# Apache Camel dataset
camel4 = pd.read_csv('data/camel-1.4.csv')
camel4["defective"] = (camel4["bug"] > 0).astype(int)
camel4.drop(['name', 'version', 'name.1', 'bug'], axis=1, inplace=True)
camel4.head()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,defective
0,8,2,0,8,31,28,1,7,1,2.000000,...,0.000000,0,0.588235,0.347222,1,1,48.250000,7,3.1250,1
1,5,0,0,9,17,10,2,8,2,2.000000,...,0.000000,0,0.000000,0.450000,0,0,18.400000,2,1.0000,0
2,1,1,0,0,1,0,0,0,1,2.000000,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
3,13,3,0,41,73,50,5,41,5,0.892857,...,0.714286,0,0.645161,0.250000,1,1,24.923077,5,1.6154,0
4,7,2,0,6,13,17,4,2,7,0.750000,...,1.000000,1,0.333333,0.428571,0,0,5.285714,1,0.8571,0


In [50]:
# Split data into 0% training and 100% test sets
y = camel4['defective'].values                  # y is response vector
X = camel4.drop('defective', axis=1).values     # X is array of predictors(features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=myseed)

In [51]:
round(lr_base.score(X_test, y_test), 3)

0.815

In [52]:
y_pred = lr_base.predict(X_test)
print(round(precision_score(y_test, y_pred), 3))
print(round(recall_score(y_test, y_pred), 3))
print(round(f1_score(y_test, y_pred), 3))
print(round(average_precision_score(y_test, y_pred), 3))
print(round(roc_auc_score(y_test, y_pred), 3))

0.167
0.028
0.048
0.167
0.5


## References

  1. Müller, Andreas C., and Sarah Guido. *Introduction to machine learning with Python: a guide for data scientists*. O'Reilly Media, Inc., 2016.
  2. Rodriguez-Mier, Pablo. A tutorial on Differential Evolution with Python. https://pablormier.github.io/2017/09/05/a-tutorial-on-differential-evolution-with-python/. 2017.
  2. Scikit-learn documentation. https://scikit-learn.org/stable/modules/grid_search.html. 2019.